In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Sample data
df = pd.read_csv('10k_single_agent.csv')

# Encode agent_id to numerical values
agent_encoder = LabelEncoder()
df['agent_encoded'] = agent_encoder.fit_transform(df['agent_id'])

# Encode moves into numerical format
df[['row', 'col']] = df['context'].str.split(',', expand=True).astype(int)

# Convert to sequences
sequence_length = len(df)  # Use the full length of the sequence
num_features = 3  # agent_encoded + row + col

# Initialize sequences and targets
sequences = np.zeros((1, sequence_length, num_features), dtype=float)

for i, row in df.iterrows():
    sequences[0, i, 0] = row['agent_encoded']
    sequences[0, i, 1] = row['row']
    sequences[0, i, 2] = row['col']


ValueError: invalid literal for int() with base 10: 'New Game'

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the CNN model
cnn_model = Sequential([
    Conv2D(32, (2, 2), activation='relu', input_shape=(board_size, board_size, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (2, 2), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Assuming binary classification for simplicity
])

cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Prepare labels (dummy labels for this example)
labels = np.random.randint(0, 2, size=(len(board_states),))

# Train the CNN model
cnn_model.fit(board_states[..., np.newaxis], tf.keras.utils.to_categorical(labels), epochs=5)


In [ ]:
feature_extractor = tf.keras.Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

# Extract features
train_features = feature_extractor.predict(board_states[..., np.newaxis])


In [ ]:
import xgboost as xgb

# Train XGBoost model
xgb_model = xgb.XGBClassifier()
xgb_model.fit(train_features, labels)


In [ ]:
def ensemble_predict(board_state):
    features = feature_extractor.predict(board_state[np.newaxis, ..., np.newaxis])
    return xgb_model.predict(features)

# Predict on new data
predictions = ensemble_predict(board_states[0])
print(predictions)
